In [28]:
%pwd

'd:\\Projects\\chicken-disease-classification'

Incase the pwd is not the home folder, change the directory to home folder
import os
os.chdir("../")

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [30]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfiguratonManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):
        print("Configuration Manager Initiated")
        # print(f"Config File Path: {config_filepath}, with dtype {type(config_filepath)}")
        # print(f"Params File Path: {params_filepath} with dtype {type(params_filepath)}")
        self.configs = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # print(f"Configs: {self.configs}")
        create_directories([self.configs.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.configs.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

# config_dummy = ConfiguratonManager()


In [32]:
import os
import urllib.request as request
from zipfile import ZipFile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [33]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file with the folowing headers: \n{headers}")
        
        else:
            logger.info(f"File already exists at {self.config.local_data_file} of size {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zip:
            zip.extractall(path=unzip_path)
            logger.info(f"Extracted file at {self.config.unzip_dir}")
                

In [34]:
try:
    config = ConfiguratonManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e
    

Configuration Manager Initiated
Config File Path: D:\Projects\chicken-disease-classification\config\config.yaml, with dtype <class 'pathlib.WindowsPath'>
Params File Path: D:\Projects\chicken-disease-classification\params.yaml with dtype <class 'pathlib.WindowsPath'>
[ 2023-09-14 01:27:02,546: INFO: common:  yaml file: D:\Projects\chicken-disease-classification\config\config.yaml loaded successfully]
[ 2023-09-14 01:27:02,550: INFO: common:  yaml file: D:\Projects\chicken-disease-classification\params.yaml loaded successfully]
Configs: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/QTanweer/Chicken-Disease-Dataset/raw/main/chicken_disease.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion/Chicken-Disease-Dataset'}}
[ 2023-09-14 01:27:02,553: INFO: common:  directory: artifacts created successfully]
[ 2023-09-14 01:27:02,555: INFO: common:  directory: artifacts

[ 2023-09-14 01:28:02,734: INFO: 3909421219:  Downloaded file with the folowing headers: 
Connection: close
Content-Length: 24046553
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d78fcf292e31630cad7071fd7f1e4dc9b22a2822f11e105ce7c4c439da739169"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AE86:2695:25A173:2D1059:650212CD
Accept-Ranges: bytes
Date: Wed, 13 Sep 2023 20:27:04 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990027-FJR
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1694636823.903816,VS0,VE1125
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 3556bba28035445c18c81a99b7e26f0843e95895
Expires: Wed, 13 Sep 2023 20:32:04 GMT
Source-Age: 0

]
[ 2023-09-14 01:28:05,950: INFO: 3909421219:  Extracted file at